In [56]:
pip install scikit-learn nltk 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords

## Database Connection

In [58]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [59]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [60]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [61]:
descripcion_novedad = pd.read_sql_table('dim_descripcion_novedad', etl_conn) 
novedad_servicio = pd.read_sql_table("mensajeria_novedadesservicio",url_db)

In [62]:
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7


## Transformation

In [63]:
num_clusters = 10  # Número original de grupos
extra_group_name = "Grupo Especial"  # Nombre para el nuevo grupo

# Definir una lista de stopwords en español
stopwords_spanish = [
    'yo', 'tú', 'él', 'ella', 'usted', 'nosotros', 'vosotros', 'ellos',
    'la', 'el', 'al', 'los', 'las', 'de', 'y', 'en', 'que', 'a', 'es', 'un', 
    'una', 'por', 'para', 'con', 'no', 'si', 'se', 'me', 'te', 'le', 
    'lo', 'la', 'sus', 'mi', 'su', 'esto', 'eso', 'aquello', 'este', 
    'esa', 'aquella', 'ser', 'haber', 'estar', 'tener', 'hacer', 
    'poder', 'decir', 'ir', 'ver', 'dar', 'saber', 'querer', 'llegar',
    'pasar', 'deber', 'poner', 'parecer', 'quedar', 'creer', 'hablar', 
    'llevar', 'dejar', 'seguir', 'encontrar', 'llamar', 'venir', 
    'pensar', 'salir', 'volver', 'tomar', 'conocer', 'vivir', 
    'sentir', 'tratar', 'mirar', 'contar', 'empezar', 'esperar',
    'buscar', 'existir', 'entrar', 'trabajar', 'escribir', 
    'perder', 'producir', 'ocurrir', 'entender', 'poder', 'probar',
    'saber', 'así', 'mientras', 'cualquier', 'algunos', 'muchos', 
    'todos', 'cada', 'pocos', 'nada', 'todo', 'b3', 'b66', 'ya', 'qué'
]

# Separar documentos especiales y normales
special_documents = []
normal_documents = []

for doc in novedad_servicio['descripcion']:
    # Verificar si el documento es una letra o una sola palabra
    if len(doc.strip()) == 1 or (len(doc.strip().split()) == 1):
        special_documents.append(doc)
    else:
        normal_documents.append(doc)

# Combinar los documentos normales y especiales
documents = normal_documents + special_documents

# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words=stopwords_spanish)
X = vectorizer.fit_transform(documents)

# Instanciar y ajustar el modelo K-Means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Obtener las etiquetas de los clusters
labels = kmeans.labels_

# Agregar etiquetas al DataFrame original
novedad_servicio['grupo'] = None  # Inicializar una nueva columna para los grupos

# Asignar las etiquetas a las descripciones en el DataFrame
for i, doc in enumerate(novedad_servicio['descripcion']):
    if len(doc.strip()) == 1 or (len(doc.strip().split()) == 1):
        novedad_servicio.at[i, 'grupo'] = 10  # Asignar al grupo especial
    else:
        novedad_servicio.at[i, 'grupo'] = labels[i - len(special_documents)]  # Ajustar el índice para documentos normales

In [64]:
# Mostrar el DataFrame con las descripciones y su clasificación
novedad_servicio.head(5)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,grupo
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,10
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,10
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,10
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,10
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,10


In [65]:
hecho_novedad_servicio = novedad_servicio[["id", "descripcion", "grupo"]]
hecho_novedad_servicio = pd.merge(hecho_novedad_servicio, descripcion_novedad[["key_descripcion_novedad"]], left_on="grupo", right_on="key_descripcion_novedad", how="left")
hecho_novedad_servicio

,id,descripcion,grupo,key_descripcion_novedad
0,4,A,10,10
1,5,Halo,10,10
2,6,A,10,10
3,7,B,10,10
4,8,A,10,10
...,...,...,...,...
5203,5246,"Facturaron el refrigerante equivocado, se hará...",9,9
5204,5247,Edte drrvicio lo hace angelo,9,9
5205,5248,Edte lo hace csrlos,9,9
5206,5249,Este lohace csrlos,9,9


In [73]:
melo = novedad_servicio.groupby("grupo").size().reset_index(name="cantidad_por_descripcion_novedad")
melo
nov = novedad_servicio[novedad_servicio["grupo"] == 6]
nov.head(15)

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,grupo
158,164,2024-02-16 00:24:09.451142+00:00,1,En colombia no tiene conocimiento de unidades ...,1321,True,12,6
159,165,2024-02-16 00:55:39.126358+00:00,1,Se recoge una mx de orina adicional,1324,True,12,6
171,177,2024-02-16 20:34:07.153788+00:00,1,En camino a la entrega el cliente me llama e i...,1404,True,11,6
178,184,2024-02-17 14:38:32.425110+00:00,1,"2da, no han bajado los filtros\nAsunto, el téc...",1449,True,8,6
205,211,2024-02-21 17:23:28.062677+00:00,1,Cerrado en taller cars habren a la 1 30,1778,True,25,6
234,239,2024-02-23 17:19:05.796591+00:00,1,No actualiza y se reportará por los grupos de ...,2053,True,31,6
238,243,2024-02-23 21:29:44.544064+00:00,1,Se recogió a las 15:46 fallo en la aplicación,2130,True,15,6
239,244,2024-02-24 03:11:34.303559+00:00,2,Los componentes no estaban listos ellos llaman...,2165,True,32,6
285,289,2024-02-28 13:40:49.228479+00:00,2,El stop no esta listo para recojer enterado je...,2559,True,25,6
290,294,2024-02-28 18:35:15.019325+00:00,2,El servicio está mal redactado no puedo contin...,2620,True,29,6


In [67]:
descripcion_novedad

,key_descripcion_novedad,grupo,terminos_clave,descripcion
0,0,Grupo 0,"está, informa, listo, cerrado, servicio, aseso...",Asistencia y Confirmación de Servicios
1,1,Grupo 1,"espera, están, sigo, entreguen, terminal, reci...",Logística de Entrega de Encomiendas
2,2,Grupo 2,"pedido, despachen, esperando, espera, entregue...",Seguimiento de Pedidos y Despachos
3,3,Grupo 3,"toma, compañero, otro, asunto, trasbordo, serv...",Comunicación y Gestión Colaborativa
4,4,Grupo 4,"servicio, camioneta, mal, programado, persona,...",Desafíos en la Programación de Servicios
5,5,Grupo 5,"compañero, otro, servicio, entrego, continúa, ...",Continuidad en la Comunicación y Entrega
6,6,Grupo 6,"enterado, pendiente, ok, recoger, salgo, aseso...",Confirmación y Seguimiento de Acciones
7,7,Grupo 7,"deja, repuesto, escritorio, yulieth, mañana, s...",Gestión de Recursos y Repuestos
8,8,Grupo 8,"esperando, despacho, bodega, entrega, reciban,...",Coordinación de Entregas y Solicitudes
9,9,Grupo 9,"repuesto, entrega, hay, informa, encuentra, se...",Intercambio de Información y Gestión Documental


In [68]:
# Contar solicitudes por cada 'key_descripcion_novedad'
cantidad_por_descripcion_novedad = hecho_novedad_servicio.groupby("key_descripcion_novedad").size().reset_index(name="cantidad_por_descripcion_novedad")
# Hacer merge para agregar la columna 'cantidad_por_descripcion_novedad' al DataFrame original
hecho_novedad_servicio = hecho_novedad_servicio.merge(cantidad_por_descripcion_novedad, on="key_descripcion_novedad", how="left")
# Eliminar columnas innecesarias
hecho_novedad_servicio.drop(columns=["descripcion", "grupo"], inplace=True)
# Mostrar las primeras 10 filas del DataFrame resultante
hecho_novedad_servicio.head(10)

,id,key_descripcion_novedad,cantidad_por_descripcion_novedad
0,4,10,108
1,5,10,108
2,6,10,108
3,7,10,108
4,8,10,108
5,9,10,108
6,10,10,108
7,11,10,108
8,12,10,108
9,15,9,2191


In [69]:
print(len(hecho_novedad_servicio))

5208


## Loading

In [70]:
hecho_novedad_servicio.to_sql("hecho_novedad_servicio", etl_conn, if_exists="replace", index_label="key_novedad_servicio") 

208